In [14]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
# !pip install python-dotenv

In [5]:
from dotenv import load_dotenv
load_dotenv()

False

In [ ]:
# !pip install pandas

In [4]:
import pandas as pd

file_path = ('/Users/apitchai002/Documents/Clones/nit-patna-genai-2025/Automobile.csv') # insert the path of the csv file
data = pd.read_csv(file_path)

#preview the csv file
data.head()

,name,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,70,usa
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,70,usa
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,70,usa
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,70,usa
4,ford torino,17.0,8,302.0,140.0,3449,10.5,70,usa


In [ ]:
# !pip install langchain-community

In [9]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path=file_path)
doc_splits = loader.load_and_split()

In [ ]:
# !pip install langchain-google-genai
# !pip install chromadb

In [18]:
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

load_dotenv()

vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=GoogleGenerativeAIEmbeddings(model='models/embedding-001'),
    persist_directory="chroma_vdb"
)

In [19]:
retriever = vectorstore.as_retriever()

In [20]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_csv_file",
    "Search and return information about crimes from the csv files.",
)

In [35]:
question='how many cylinders does ford maverick with model year 75 have?'

In [27]:
relevant_text = retriever_tool.invoke(question)
print(relevant_text)

name: ford maverick
mpg: 15
cylinders: 6
displacement: 250
horsepower: 72
weight: 3158
acceleration: 19.5
model_year: 75
origin: usa

name: ford maverick
mpg: 24
cylinders: 6
displacement: 200
horsepower: 81
weight: 3012
acceleration: 17.6
model_year: 76
origin: usa

name: ford maverick
mpg: 18
cylinders: 6
displacement: 250
horsepower: 88
weight: 3021
acceleration: 16.5
model_year: 73
origin: usa

name: ford maverick
mpg: 21
cylinders: 6
displacement: 200
horsepower: 
weight: 2875
acceleration: 17
model_year: 74
origin: usa


In [46]:
# !pip install langchain-huggingface
# !pip install transformers
# !pip install numpy

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [56]:
from langchain.prompts import PromptTemplate

template = """Answer the question in one line using the following information:

```{information}```.


*** Question ***

{question}

*** Answer ***"""

prompt = PromptTemplate.from_template(template=template)

In [57]:
prompt_formatted_str: str = prompt.format(question=question, information=relevant_text)

In [58]:
prompt_formatted_str

'Answer the question in one line using the following information:\n\n```name: ford maverick\nmpg: 15\ncylinders: 6\ndisplacement: 250\nhorsepower: 72\nweight: 3158\nacceleration: 19.5\nmodel_year: 75\norigin: usa\n\nname: ford maverick\nmpg: 24\ncylinders: 6\ndisplacement: 200\nhorsepower: 81\nweight: 3012\nacceleration: 17.6\nmodel_year: 76\norigin: usa\n\nname: ford maverick\nmpg: 18\ncylinders: 6\ndisplacement: 250\nhorsepower: 88\nweight: 3021\nacceleration: 16.5\nmodel_year: 73\norigin: usa\n\nname: ford maverick\nmpg: 21\ncylinders: 6\ndisplacement: 200\nhorsepower: \nweight: 2875\nacceleration: 17\nmodel_year: 74\norigin: usa```.\n\n\n*** Question ***\n\nhow many cylinders does ford maverick with model year 75 have?\n\n*** Answer ***'

In [ ]:
response = llm.invoke(prompt_formatted_str)

In [61]:
print(response.content)

The Ford Maverick with model year 75 has 6 cylinders.
